In [1]:
import sqlite3
import nltk
import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import spacy
from itertools import chain

/home/ai-user/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conn = sqlite3.connect ("job.db")
print("Opened database successfully")

c = conn.cursor()   #获取游标
sql = "SELECT title, location, telecommuting, employment_type, required_experience, required_education, function, industry from 'FAKE_JOB_POSTINGS'"
cursor = c.execute(sql)#执行sql语句

title, location, telecommuting, employment_type, required_experience, required_education, function, industry = ([] for i in range(8))

for row in cursor:
    title.append(row[0])
    location.append(row[1])
    telecommuting.append(row[2])
    employment_type.append(row[3])
    required_experience.append(row[4])
    required_education.append(row[5])
    function.append(row[6])
    industry.append(row[7])

    
conn.close()        #关闭数据库连接


#print(benefits[1])

Opened database successfully


In [3]:
def data(i):
    print(title[i])
    print(location[i])
    print(type(telecommuting[i]))
    print(employment_type[i])
    print(required_experience[i])
    print(required_education[i])
    print(function[i])
    
data(1)

Customer Service - Cloud Video Production
NZ, , Auckland
<class 'int'>
Full-time
Not Applicable
None
Customer Service


In [4]:
a = set(industry)
# a

In [5]:
find1 = '1'
a=title[128].find(find1)
print(title[128])
print(a)
if a == -1:
    print('remain')
else:
    print('delet')


Precision Ag Intern Spring 2015 $2000 Per Month
29
delet


In [6]:
# 1.较为理想的工作地点
# {'AE', 'AU', 'CA', 'DE', 'EG', 'GB', 'GR', 'IL', 'IN', 'NZ', None, 'PK', 'PL', 'SG', 'US'}
# A.北美发达国家 'CA', 'US'
# B.欧洲发达国家  'DE', 'GB', 'GR', 'PL'
# C.大洋洲发达国家  'AU', 'NZ'
# D.亚洲发达国家 'IL', 'SG'
# E.发展中国家 'AE',  'EG',  'IN',  'PK'
# F. All
# A-1,B-2,C-3,D-4,E-5

In [7]:
def Country_Index(m):
    if m == None:
        country = None
    else:
        country = m[:2]
    #print(country)
    if country == None:
        index = 0
    elif country == 'US' or country == 'CA':
        index = 1
    elif country == 'DE' or country == 'GB' or country == 'GR' or country == 'PL':
        index = 2
    elif country == 'AU' or country == 'NZ':
        index = 3
    elif country == 'IL' or country == 'SG':
        index = 4
    elif country == 'AE' or country == 'EG' or country == 'IN' or country == 'PK':
        index = 5
    else:
        index = 0
        
    return index

In [8]:
a=Country_Index(location[6])
a

2

In [9]:
# 2.希望得到怎样类型的工作
# A.全职有合约  'Contract', 'Full-time'
# B.兼职实习  'Part-time', 'Temporary', None, 'Other'
# C.随意  All
# A-0 B-1
def Type_Index(m):
    if m == None:
        index = 1
    elif m == 'Contract' or m == 'Full-time':
        index = 0
    elif m == 'Other' or m == 'Part-time' or m == 'Temporary':
        index = 1
    else:
        index = 1
    return index

In [10]:
a=Type_Index(employment_type[2])
print(employment_type[0:6])
a

['Other', 'Full-time', None, 'Full-time', 'Full-time', None]


1

In [11]:
# 1.相关行业工作经验
# A.有管理经验  'Executive', 'Director'
# B.有长期工作经历  'Mid-Senior level'
# C.有短期工作经历  'Entry level', 'Internship', 'Associate'
# D.无  None, 'Not Applicable'
# data storage A-3 B-2 C-1 D-0
# selection A-all B-<3 C-<2 D-0
def Experience_Index(m):
    if m == None:
        index = 0
    elif m == 'Not Applicable':
        index = 0
    elif m == 'Entry level' or m == 'Internship' or m == 'Associate':
        index = 1
    elif m == 'Mid-Senior level':
        index = 2
    elif m == 'Executive' or m == 'Director':
        index = 3
    else:
        index = 0
        
    return index

In [12]:
a=Experience_Index(required_experience[3])
print(required_experience[0:5])
a

['Internship', 'Not Applicable', None, 'Mid-Senior level', 'Mid-Senior level']


2

In [13]:
# 2.最高受教育程度
# A.研究生及以上 'Associate Degree', 'Doctorate', "Master's Degree", 'Professional'
# B.本科毕业 'Certification',  "Bachelor's Degree"
# C.专科 'Vocational', 'Vocational - Degree', 'Vocational - HS Diploma'}
# D.高中及以下  'High School or equivalent', 'Some College Coursework Completed', 'Some High School Coursework', 'Unspecified'
# data storage A-3 B-2 C-1 D-0
# selection A-all B-<3 C-<2 D-0
def Education_Index(m):
    if m == None:
        index = 0
    elif m == 'High School or equivalent' or m == 'Some College Coursework Completed' or m == 'Some High School Coursework' or m == 'Unspecified':
        index = 0
    elif m == 'Vocational' or m == 'Vocational - Degree' or m == 'Vocational - HS Diploma':
        index = 1
    elif m == 'Certification' or m == "Bachelor's Degree":
        index = 2
    elif m == 'Associate Degree' or m == 'Doctorate' or m == "Master's Degree" or m == 'Professional':
        index = 3
    else:
        index = 0
        
    return index

In [14]:
a=Education_Index(required_education[9])
print(required_education[0:10])
a

[None, None, None, "Bachelor's Degree", "Bachelor's Degree", None, "Master's Degree", None, None, 'High School or equivalent']


0

In [15]:
# 3.专业方向
# A.管理 EXTJ
# B.工程 ISTJ
# C.法学 XSXJ
# D.艺术 IXFP
# E.服务 ESFX
# F.经济 XXTP
# G.理学 INTX
# H.传播 XNFP
# I.医学 XSFJ
# J.教育 EXFJ
# K.语言文学 INFP
# L.其他 XXXX
# A-0,B-1,C-2,D-3,E-4,F-5,G-6,H-7,I-8,J-9,K-10,L-11

In [16]:
def Main_Index(m):
    management = ['Accounting','Business Supplies and Equipment','Consumer Electronics','Education Management','Executive Office','Government Administration',
              'Government Relations','Human Resources','Investment Management','Logistics and Supply Chain','Management Consulting','Market Research','Marketing and Advertising',
              'Nonprofit Organization Management','Outsourcing/Offshoring','Package/Freight Delivery','Public Policy','Public Safety','Security and Investigations',
              'Staffing and Recruiting','Utilities','Warehousing','Wholesale']
    
    engineering = ['Airlines/Aviation','Automotive','Architecture & Planning','Aviation & Aerospace','Building Materials','Civil Engineering','Computer & Network Security',
             'Computer Games','Computer Hardware','Computer Networking','Computer Software', 'Construction','Defense & Space','E-Learning','Electrical/Electronic Manufacturing',
             'Furniture','Industrial Automation','Internet','Libraries','Machinery','Mechanical or Industrial Engineering','Nanotechnology','Oil & Energy','Wireless',
             'Telecommunications','Program Development','Semiconductors','Shipbuilding','Textiles','Transportation/Trucking/Railroad']
    
    law = ['Alternative Dispute Resolution','Civic & Social Organization','Law Enforcement','Law Practice']

    art = ['Animation','Apparel & Fashion','Design','Graphic Design','Motion Pictures and Film','Music','Performing Arts','Photography']

    service = ['Consumer Services','Environmental Services','Events Services','Facilities Services','Financial Services','Food & Beverages',
           'Individual & Family Services','Information Services','Information Technology and Services','Legal Services','Restaurants','Retail','Sporting Goods']

    economy = ['Banking','Capital Markets','Consumer Goods','Fund-Raising','Import and Export','International Trade and Development','Insurance','Investment Banking',
           'Real Estate','Commercial Real Estate','Venture Capital & Private Equity']

    philosophy = ['Biotechnology','Chemicals','Research','Renewables & Environment']
    
    transmission = ['Broadcast Media','Entertainment','Leisure, Travel & Tourism','Media Production','Online Media','Public Relations and Communications','Publishing']
    
    medicine = ['Cosmetics','Hospital & Health Care','Hospitality','Health, Wellness and Fitness','Medical Devices','Medical Practice',
                'Mental Health Care','Pharmaceuticals','Veterinary']
    
    education = ['Higher Education','Primary/Secondary Education','Professional Training & Coaching']
    
    literature = ['Writing and Editing','Translation and Localization']
    
    others = ['Gambling & Casinos','Luxury Goods & Jewelry','Maritime','Museums and Institutions',None,'Packaging and Containers','Wine and Spirits','Plastics','Printing',
          'Philanthropy','Religious Institutions','Military','Mining & Metals','Sports','Farming','Fishery','Food Production','Ranching']

    
    if m in management:
        index = 0
        index_EI = 'E'
        index_NS = 'X'
        index_TF = 'T'
        index_PJ = 'J'
    elif m in engineering:
        index = 1
        index_EI = 'I'
        index_NS = 'S'
        index_TF = 'T'
        index_PJ = 'J'
    elif m in law:
        index = 2
        index_EI = 'X'
        index_NS = 'S'
        index_TF = 'X'
        index_PJ = 'J'
    elif m in art:
        index = 3
        index_EI = 'I'
        index_NS = 'X'
        index_TF = 'F'
        index_PJ = 'P'
    elif m in service:
        index = 4
        index_EI = 'E'
        index_NS = 'S'
        index_TF = 'F'
        index_PJ = 'X'
    elif m in economy:
        index = 5
        index_EI = 'X'
        index_NS = 'X'
        index_TF = 'T'
        index_PJ = 'P'
    elif m in philosophy:
        index = 6
        index_EI = 'I'
        index_NS = 'N'
        index_TF = 'T'
        index_PJ = 'X'
    elif m in transmission:
        index = 7
        index_EI = 'X'
        index_NS = 'N'
        index_TF = 'F'
        index_PJ = 'P'
    elif m in medicine:
        index = 8
        index_EI = 'X'
        index_NS = 'S'
        index_TF = 'F'
        index_PJ = 'J'
    elif m in education:
        index = 9
        index_EI = 'E'
        index_NS = 'X'
        index_TF = 'F'
        index_PJ = 'J'
    elif m in literature:
        index = 10
        index_EI = 'I'
        index_NS = 'N'
        index_TF = 'F'
        index_PJ = 'P'
    else:
        index = 11
        index_EI = 'X'
        index_NS = 'X'
        index_TF = 'X'
        index_PJ = 'X'
    return index,index_EI,index_NS,index_TF,index_PJ

In [17]:
a=Main_Index(industry[9])[1]
print(industry[0:10])
a

[None, 'Marketing and Advertising', None, 'Computer Software', 'Hospital & Health Care', None, 'Online Media', None, 'Information Technology and Services', 'Financial Services']


'E'

In [19]:
# def destription(i,nlp_tool):
#     feature1 = preprocessing(description[i], nlp_tool)
#     if requirements[i] == None:
#         feature2 = []
#     else:
#         feature2 = preprocessing(requirements[i], nlp_tool)
#     if benefits[i] == None:
#         feature3 = []
#     else:
#         feature3 = preprocessing(benefits[i], nlp_tool)
#     full_list = list(chain(feature1,feature2,feature3))
#     full_list = ",".join(full_list)
#     return full_list

In [20]:
conn = sqlite3.connect ("job.db")
print("Opened database successfully")

c = conn.cursor() 
# 执行sql创建表
sql = 'CREATE TABLE "test5" ("job_id"INTEGER,"title"TEXT,"telecommuting"INTEGER,"country_id"INTEGER,"type_id"INTEGER,"experience_id"INTEGER,"education_id"INTEGER,"main_id"INTEGER,"mbti_id_1"TEXT,"mbti_id_2"TEXT,"mbti_id_3"TEXT,"mbti_id_4"TEXT)'
try:
    conn.execute(sql)
except Exception as e:
    print(e)
    print('创建表失败')
        # 关闭游标
    conn.close()
        # 关闭连接
    conn.close()

Opened database successfully


In [21]:
conn = sqlite3.connect ("job.db")
print("Opened database successfully")

c = conn.cursor() 
sql = 'insert into test5(job_id, title, telecommuting,country_id,type_id,experience_id,education_id,main_id,mbti_id_1,mbti_id_2,mbti_id_3,mbti_id_4) values(?,?,?,?,?,?,?,?,?,?,?,?)'
i=0
find1 = '1'

while i < 17880:
    a=title[i].find(find1)
    if a == -1:
        country_id = Country_Index(location[i])
        type_id = Type_Index(employment_type[i])
        experience_id = Experience_Index(required_experience[i])
        education_id = Education_Index(required_education[i])
        main_id = Main_Index(industry[i])[0]
        mbti_id_1 = Main_Index(industry[i])[1]
        mbti_id_2 = Main_Index(industry[i])[2]
        mbti_id_3 = Main_Index(industry[i])[3]
        mbti_id_4 = Main_Index(industry[i])[4]
        try:
            conn.execute(sql,(i+1,title[i],telecommuting[i],country_id,type_id,experience_id,education_id,main_id,mbti_id_1,mbti_id_2,mbti_id_3,mbti_id_4))
        #提交事务
            conn.commit()
            print('插入成功')
        except Exception as e:
            print(e)
            print('插入失败')
        conn.rollback()
    else:
        print('跳过')
    i=i+1
        # 关闭游标
conn.close()
        # 关闭连接
conn.close()


Opened database successfully
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
跳过
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
跳过
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
跳过
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
跳过
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
跳过
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
跳过
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
跳过
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
跳过
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入成功
插入